Simplified Self Attention

In [1]:
import torch

inputs = torch.tensor([
    [0.43,0.15,0.89], #"Your"  x^1
    [0.55,0.87,0.66], #"Journey" x^2
    [0.57,0.85,0.64], #"Starts"  x^3
    [0.22,0.58,0.33], #"with"  x^4
    [0.77,0.25,0.10], #"one"  x^5
    [0.05,0.80,0.55] #"step"  x^6 
])

In [5]:
input_query = inputs[1]
input_query

tensor([0.5500, 0.8700, 0.6600])

In [6]:
input_1 = inputs[0]
input_1

tensor([0.4300, 0.1500, 0.8900])

In [7]:
0.5500*0.4300 +  0.8700 * 0.1500 + 0.6600*0.8900

0.9544

In [10]:
torch.dot(input_query,input_1)

tensor(0.9544)

In [14]:
i = 3
res = torch.dot(inputs[i],input_query)
res

tensor(0.8434)

In [16]:
query = inputs[1]
attn_scores = torch.empty(inputs.shape[0])
for i,x_i in enumerate(inputs):
    attn_scores[i] = torch.dot(query,x_i)
attn_scores


tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [18]:
attn_weights_tmp = attn_scores / attn_scores.sum()
attn_weights_tmp

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])

In [20]:
def softmax_naive(x):
    return torch.exp(x)/ torch.exp(x).sum(dim=0)
softmax_naive(attn_scores)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [28]:
attn_weights_2 = torch.softmax(attn_scores,dim=0)
attn_weights_2

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [32]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    #print(f"{attn_weights_2[i]} --> {inputs[i]}")
    context_vec_2 += attn_weights_2[i] * x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


Computing self attention mechanism without trainable weights

In [33]:
attn_scores = torch.empty(6,6)

for i,x_i in enumerate(inputs):
    for j,x_j in enumerate(inputs):
        attn_scores[i,j] = torch.dot(x_i,x_j)
attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [36]:
attn_scores = inputs @ inputs.T
attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [37]:
attn_weights = torch.softmax(attn_scores,dim = 1)
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [39]:
all_context_vecs = attn_weights @ inputs
all_context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

Self Attetion with trainable weights

In [40]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [42]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in,d_out))
W_key = torch.nn.Parameter(torch.rand(d_in,d_out))
W_value = torch.nn.Parameter(torch.rand(d_in,d_out))


In [43]:
W_query

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]], requires_grad=True)

In [44]:
W_key

Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]], requires_grad=True)

In [45]:
query_2 = x_2 @ W_query
query_2

tensor([0.4306, 1.4551], grad_fn=<SqueezeBackward4>)

In [47]:
keys = inputs @ W_key
values = inputs @ W_value

In [48]:
keys

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]], grad_fn=<MmBackward0>)

In [49]:
values.shape

torch.Size([6, 2])

In [50]:
keys_2 = keys[1]
attn_score_22 = torch.dot(query_2,keys_2)
attn_score_22

tensor(1.8524, grad_fn=<DotBackward0>)

In [52]:
attn_scores_2 = query_2 @ keys.T
attn_scores_2

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
       grad_fn=<SqueezeBackward4>)

In [53]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5,dim=-1)
attn_weights_2

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
       grad_fn=<SoftmaxBackward0>)

In [54]:
attn_weights_2.sum()

tensor(1., grad_fn=<SumBackward0>)

In [55]:
context_vec_2 = attn_weights_2 @ values
context_vec_2

tensor([0.3061, 0.8210], grad_fn=<SqueezeBackward4>)

Self attention class

In [57]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self,d_in,d_out):
        super().__init__()
        self.W_query = torch.nn.Parameter(torch.rand(d_in,d_out))
        self.W_key = torch.nn.Parameter(torch.rand(d_in,d_out))
        self.W_value = torch.nn.Parameter(torch.rand(d_in,d_out))
        self.d_k = self.W_key.shape[1]

    def forward(self,x):
        queries = x @ self.W_query
        keys = x @ self.W_key
        values = x @ self.W_value
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / d_k**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in,d_out)
sa_v1(inputs)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)

Better way of implementation

In [59]:
import torch.nn as nn
class SelfAttention_v2(nn.Module):
    def __init__(self,d_in,d_out,qkv_bias = False):
        super().__init__()
        self.W_query = torch.nn.Linear(d_in,d_out,bias = qkv_bias)
        self.W_key = torch.nn.Linear(d_in,d_out,bias = qkv_bias)
        self.W_value = torch.nn.Linear(d_in,d_out,bias = qkv_bias)
        self.d_k = d_out

    def forward(self,x):
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / d_k**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v2 = SelfAttention_v2(d_in,d_out)
sa_v2(inputs)

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)

Next :Applying causal attention